El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [24]:
# Importar bibliotecas
import pandas as pd
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
# Cargar DataSet
df = pd.read_csv("datasets/car_data.csv")

In [4]:
# Visualizar datos 
display(HTML('<h1> Rusty Bargain DataFrame'))
display(HTML('<hr>'))
display(HTML('<h2> Informacion general'))
display(df.info())
display(HTML('<hr>'))
display(HTML('<h2> Muestra de tabla'))
display(df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
25031,20/03/2016 13:43,9300,sedan,2008,manual,156,c_klasse,150000,2,petrol,mercedes_benz,no,20/03/2016 00:00,0,52134,02/04/2016 15:47
192716,11/03/2016 05:36,699,small,1997,manual,50,micra,150000,4,petrol,nissan,NaN,11/03/2016 00:00,0,59067,22/03/2016 17:47
136594,09/03/2016 22:57,2500,coupe,1998,manual,197,clk,150000,12,petrol,mercedes_benz,no,09/03/2016 00:00,0,35066,30/03/2016 22:17
326323,12/03/2016 21:53,9750,sedan,2005,auto,275,a8,150000,6,gasoline,audi,no,12/03/2016 00:00,0,89231,03/04/2016 19:47
201290,01/04/2016 19:39,350,small,1995,manual,75,3_reihe,150000,3,petrol,peugeot,NaN,01/04/2016 00:00,0,97980,03/04/2016 16:17


In [10]:
# Revisar la cantidad de valores nulos en cada columna
missing_values = df.isnull().sum()

# Calcular el porcentaje de valores nulos
missing_percent = (missing_values / len(df)) * 100

# Resultados de valores faltantes
missing_data = pd.DataFrame({
    'Total Missing': missing_values,
    'Percent Missing (%)': missing_percent
})

# Filtrar columnas que tengan valores nulos
missing_data = missing_data[missing_data['Total Missing'] > 0].sort_values(by='Percent Missing (%)', ascending=False)

display(missing_data)


,Total Missing,Percent Missing (%)
NotRepaired,71154,20.079070
VehicleType,37490,10.579368
FuelType,32895,9.282697
Gearbox,19833,5.596709
Model,19705,5.560588


In [17]:
# Filtrar filas con valores nulos en ciertas columnas
filtered_df = df.dropna(subset=['VehicleType', 'FuelType', 'Gearbox', 'Model']).copy()

# Imputar la columna 'NotRepaired' con la categoría 'desconocido' usando .loc
filtered_df.loc[:, 'NotRepaired'] = filtered_df['NotRepaired'].fillna('desconocido')

# Verificar si quedan valores nulos
remaining_missing = filtered_df.isnull().sum()

# Mostrar las columnas que todavía tienen valores nulos


if not remaining_missing[remaining_missing > 0].empty:
    print("Aún hay valores nulos en las siguientes columnas:", remaining_missing[remaining_missing > 0])
else:
    print("No hay más valores nulos en el DataFrame.")

display(filtered_df.sample(5))


No hay más valores nulos en el DataFrame.


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
180053,19/03/2016 15:54,1050,small,2002,manual,131,ibiza,150000,3,gasoline,seat,desconocido,19/03/2016 00:00,0,82178,21/03/2016 21:17
259675,23/03/2016 18:53,13490,coupe,2010,manual,143,1er,125000,9,gasoline,bmw,no,23/03/2016 00:00,0,73329,05/04/2016 23:45
279033,13/03/2016 19:43,999,convertible,1994,manual,75,golf,150000,8,petrol,volkswagen,no,13/03/2016 00:00,0,71063,06/04/2016 18:44
305143,11/03/2016 16:43,11900,sedan,2008,auto,387,e_klasse,150000,1,petrol,mercedes_benz,no,11/03/2016 00:00,0,59192,15/03/2016 08:15
98347,22/03/2016 20:51,8340,convertible,2004,auto,192,3er,150000,7,petrol,bmw,no,22/03/2016 00:00,0,41540,06/04/2016 14:47


### Preparacion de datos (Explicacion)

Durante este proceso de preparacion de datos se realizaron varias cosas, por ejemplo
- Se cargo el archivo car_data.csv en un dataframe y realizamos un analisis de su estructura
- Identificamos las columnas con valores nulos, cuantificando los porcentajes para determinar la mejor estrategia de manejo.

Preparacion de datos

- Se eliminaron las filas con valores nulos en VehicleType, FuelType, Gearbox y Model.
- Para la columna NotRepaired, imputamos los valores nulos con una categoría "desconocido" para mantener la integridad de los datos.
- Verificamos si quedaban valores nulos en otras columnas tras la imputación.

## Entrenamiento del modelo 

In [21]:
df_clean = filtered_df.copy()
# Mostrar las columnas del DataFrame
# Identificar las columnas categóricas y numéricas para el preprocesamiento
categorical_columns = df_clean.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df_clean.select_dtypes(exclude=['object']).columns.tolist()
numerical_columns.remove('Price')  # Asegurarse de que 'Price' no está en las columnas numéricas


In [22]:

# Preprocesador para variables categóricas usando One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocesador para variables numéricas usando escalado estándar
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combinar los transformadores en un único preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar el tamaño de los conjuntos de datos
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


((227300, 15), (56826, 15), (227300,), (56826,))

## Análisis del modelo

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos